# 1. Install and Import

In [ ]:
# run_in = 'local'
run_in = 'colab'

if run_in == 'local':
    import csv
    import os
    import pandas as pd
    import numpy as np
    import tweepy
    import datetime
    import time
    import pprint
    pp = pprint.PrettyPrinter(indent=4)
    import json
    
if run_in == 'colab':
    !pip install xlrd
    !pip install XlsxWriter
    !pip install tweepy
    import csv
    import os
    import pandas as pd
    import numpy as np
    import tweepy
    import datetime
    import time
    import pprint
    pp = pprint.PrettyPrinter(indent=4)
    import json

## <p><font color="red">Define Version number!</font></p>

# 2. Define your personal path to the folder we will work in

We expect an xlsx file with multiple sheets

In [ ]:
if run_in == 'local':
  base_path = "/users/USERNAME/Google Drive/USERNAME/"
if run_in == 'colab':
  base_path = "/content/drive/My Drive/USERNAME/"
  from google.colab import drive
  drive.mount('/content/drive')
  
print('base_path set to:')
print(base_path)

In [ ]:
def Average(lst): 
  avg = sum(lst) / len(lst)
  return avg 

def delta_survey(df):
  delta_survey_period_list = []
  for i in range(len(df)):
    company_data = df.loc[i].values
    #print("company_data", company_data)
    company_data_cleaned = [x for x in company_data if str(x) != 'nan']
    #print('company_data_cleaned', company_data_cleaned)
    try:
      first_data_point = company_data_cleaned[0]
      last_data_point = company_data_cleaned[-1]
      #print("first_data_point:", first_data_point)
      #print("last_data_point:", last_data_point)
      delta_survey_period = last_data_point - first_data_point
      #print("last_data_point - first_data_point =", delta_survey_period)
      delta_survey_period_list.append(delta_survey_period)
    except:
      delta_survey_period = 0
      delta_survey_period_list.append(delta_survey_period)
  return delta_survey_period_list

def rel_diff_survey(df):
  rel_differences_survey_period_list = []
  for i in range(len(df)):
    company_data = df.loc[i].values
    #print("company_data", company_data)
    company_data_cleaned = [x for x in company_data if str(x) != 'nan']
    #print('company_data_cleaned', company_data_cleaned)
    try:
      first_data_point = company_data_cleaned[0]
      last_data_point = company_data_cleaned[-1]
      # print("first_data_point:", first_data_point)
      # print("last_data_point:", last_data_point)
      delta_survey_period = last_data_point - first_data_point
      # print("last_data_point - first_data_point =", delta_survey_period)
      rel_difference_survey_period = delta_survey_period / Average(company_data_cleaned)
      # print("rel_difference_survey_period", rel_difference_survey_period)
      rel_differences_survey_period_list.append(rel_difference_survey_period)
    except:
      rel_difference_survey_period = 0
      rel_differences_survey_period_list.append(rel_difference_survey_period)
  return rel_differences_survey_period_list

In [ ]:
version_id = "V14"

# Import all current companies

In [ ]:
all_startup_file = os.path.join(base_path + "Pitchbook_Crunchbase_Tracxn_Raw/Don't touch V14/V14.xlsx")
print("all_startup_file_path is: " + all_startup_file + " and the file exists: " + str(os.path.isfile(all_startup_file)))            

In [ ]:
company_name_df = pd.read_excel(all_startup_file, 'V14df_all', usecols = "A", skiprows=0).drop_duplicates().reset_index(drop=True)
company_name_df.head()

#3. Import latest company version

In [ ]:
df_company_founding_dates = pd.read_excel(base_path + "Analysis/Company Founding Dates.xlsx", "Founding Dates", skiprows=0)
df_company_founding_dates = df_company_founding_dates[df_company_founding_dates.founded_date.notnull()]
df_company_founding_dates = df_company_founding_dates[["company_name","founded_date"]]
df_company_founding_dates.sort_values(by=['company_name', 'founded_date'], inplace = True)
df_company_founding_dates.drop_duplicates(subset="company_name", keep='first', inplace=True)
df_company_founding_dates.reset_index(drop=True, inplace = True)
df_company_founding_dates.head()

In [ ]:
df_company_founding_dates.describe()

In [ ]:
#df_company_founding_dates_no_nat = df_company_founding_dates.loc[df_company_founding_dates['founded_date'] != np.datetime64('NaT')]
#df_company_founding_dates_no_nat = df_company_founding_dates.loc[df_company_founding_dates['founded_date'] != str("NaT")]
df_company_founding_dates_no_nat = df_company_founding_dates[df_company_founding_dates.founded_date.notnull()]
df_company_founding_dates_no_nat.head()

# Crate timedelta

In [ ]:
df_funding_rounds = pd.read_excel(base_path + "Analysis/companies_rounds.xlsx")
df_funding_rounds.head()

In [ ]:
df_company_founding_dates.head()

In [ ]:
df_funding_rounds_and_founding_date = df_company_founding_dates.merge(df_funding_rounds, on='company_name', how='left')
df_funding_rounds_and_founding_date.head()

In [ ]:
#Search for entry
df_funding_rounds_and_founding_date_search = df_funding_rounds_and_founding_date.loc[df_funding_rounds_and_founding_date['company_name'] == "vizibl"]
df_funding_rounds_and_founding_date_search

In [ ]:
headers_date = [h for h in list(df_funding_rounds_and_founding_date) if "date" in h and "total_raised_to_date" not in h]
df_round_dates = df_funding_rounds_and_founding_date[headers_date]
df_round_dates.head()

In [ ]:
df_difference_between_dates = pd.DataFrame()
df_difference_between_dates['dt_1'] = df_funding_rounds_and_founding_date['deal_date_0'].sub(df_funding_rounds_and_founding_date['founded_date'], axis=0)
df_difference_between_dates['dt_2'] = df_funding_rounds_and_founding_date['deal_date_1'].sub(df_funding_rounds_and_founding_date['deal_date_0'], axis=0)
df_difference_between_dates['dt_3'] = df_funding_rounds_and_founding_date['deal_date_2'].sub(df_funding_rounds_and_founding_date['deal_date_1'], axis=0)
df_difference_between_dates['dt_4'] = df_funding_rounds_and_founding_date['deal_date_3'].sub(df_funding_rounds_and_founding_date['deal_date_2'], axis=0)
df_difference_between_dates['dt_5'] = df_funding_rounds_and_founding_date['deal_date_4'].sub(df_funding_rounds_and_founding_date['deal_date_3'], axis=0)
df_difference_between_dates['dt_6'] = df_funding_rounds_and_founding_date['deal_date_5'].sub(df_funding_rounds_and_founding_date['deal_date_4'], axis=0)
df_difference_between_dates['dt_7'] = df_funding_rounds_and_founding_date['deal_date_6'].sub(df_funding_rounds_and_founding_date['deal_date_5'], axis=0)
df_difference_between_dates['dt_8'] = df_funding_rounds_and_founding_date['deal_date_7'].sub(df_funding_rounds_and_founding_date['deal_date_6'], axis=0)
df_difference_between_dates['dt_9'] = df_funding_rounds_and_founding_date['deal_date_8'].sub(df_funding_rounds_and_founding_date['deal_date_7'], axis=0)
df_difference_between_dates['dt_10'] = df_funding_rounds_and_founding_date['deal_date_9'].sub(df_funding_rounds_and_founding_date['deal_date_8'], axis=0)
df_difference_between_dates['dt_11'] = df_funding_rounds_and_founding_date['deal_date_10'].sub(df_funding_rounds_and_founding_date['deal_date_9'], axis=0)
df_difference_between_dates['dt_12'] = df_funding_rounds_and_founding_date['deal_date_11'].sub(df_funding_rounds_and_founding_date['deal_date_10'], axis=0)

dt_headers = list(df_difference_between_dates)
df_difference_between_dates["dt_avg"] = df_difference_between_dates[dt_headers[:]].mean(axis = 1)

#df_difference_between_dates.sort_values('dt1',inplace = True)
df_difference_between_dates.head()

In [ ]:
df_funding_rounds_and_deltas = pd.concat([df_funding_rounds_and_founding_date, df_difference_between_dates], axis=1, sort=False)
df_funding_rounds_and_deltas.head()

In [ ]:
# Adjust dollar values to million

df_funding_rounds_and_deltas['total_raised_to_date'] = df_funding_rounds_and_deltas['total_raised_to_date']*1e6

df_funding_rounds_and_deltas['num_deal_size_0'] = df_funding_rounds_and_deltas['num_deal_size_0']*1e6
df_funding_rounds_and_deltas['num_deal_size_1'] = df_funding_rounds_and_deltas['num_deal_size_1']*1e6
df_funding_rounds_and_deltas['num_deal_size_2'] = df_funding_rounds_and_deltas['num_deal_size_2']*1e6
df_funding_rounds_and_deltas['num_deal_size_3'] = df_funding_rounds_and_deltas['num_deal_size_3']*1e6
df_funding_rounds_and_deltas['num_deal_size_4'] = df_funding_rounds_and_deltas['num_deal_size_4']*1e6
df_funding_rounds_and_deltas['num_deal_size_5'] = df_funding_rounds_and_deltas['num_deal_size_5']*1e6
df_funding_rounds_and_deltas['num_deal_size_6'] = df_funding_rounds_and_deltas['num_deal_size_6']*1e6
df_funding_rounds_and_deltas['num_deal_size_7'] = df_funding_rounds_and_deltas['num_deal_size_7']*1e6
df_funding_rounds_and_deltas['num_deal_size_8'] = df_funding_rounds_and_deltas['num_deal_size_8']*1e6
df_funding_rounds_and_deltas['num_deal_size_9'] = df_funding_rounds_and_deltas['num_deal_size_9']*1e6
df_funding_rounds_and_deltas['num_deal_size_10'] = df_funding_rounds_and_deltas['num_deal_size_10']*1e6
df_funding_rounds_and_deltas['num_deal_size_11'] = df_funding_rounds_and_deltas['num_deal_size_11']*1e6

df_funding_rounds_and_deltas['max_post_money_valuation'] = df_funding_rounds_and_deltas['max_post_money_valuation']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_0'] = df_funding_rounds_and_deltas['num_pre_money_valuation_0']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_1'] = df_funding_rounds_and_deltas['num_pre_money_valuation_1']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_2'] = df_funding_rounds_and_deltas['num_pre_money_valuation_2']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_3'] = df_funding_rounds_and_deltas['num_pre_money_valuation_3']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_4'] = df_funding_rounds_and_deltas['num_pre_money_valuation_4']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_5'] = df_funding_rounds_and_deltas['num_pre_money_valuation_5']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_6'] = df_funding_rounds_and_deltas['num_pre_money_valuation_6']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_7'] = df_funding_rounds_and_deltas['num_pre_money_valuation_7']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_8'] = df_funding_rounds_and_deltas['num_pre_money_valuation_8']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_9'] = df_funding_rounds_and_deltas['num_pre_money_valuation_9']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_10'] = df_funding_rounds_and_deltas['num_pre_money_valuation_10']*1e6
df_funding_rounds_and_deltas['num_pre_money_valuation_11'] = df_funding_rounds_and_deltas['num_pre_money_valuation_11']*1e6

df_funding_rounds_and_deltas['num_post_money_valuation_0'] = df_funding_rounds_and_deltas['num_post_money_valuation_0']*1e6
df_funding_rounds_and_deltas['num_post_money_valuation_1'] = df_funding_rounds_and_deltas['num_post_money_valuation_1']*1e6
df_funding_rounds_and_deltas['num_post_money_valuation_2'] = df_funding_rounds_and_deltas['num_post_money_valuation_2']*1e6
df_funding_rounds_and_deltas['num_post_money_valuation_3'] = df_funding_rounds_and_deltas['num_post_money_valuation_3']*1e6
df_funding_rounds_and_deltas['num_post_money_valuation_4'] = df_funding_rounds_and_deltas['num_post_money_valuation_4']*1e6
df_funding_rounds_and_deltas['num_post_money_valuation_5'] = df_funding_rounds_and_deltas['num_post_money_valuation_5']*1e6
df_funding_rounds_and_deltas['num_post_money_valuation_6'] = df_funding_rounds_and_deltas['num_post_money_valuation_6']*1e6
df_funding_rounds_and_deltas['num_post_money_valuation_7'] = df_funding_rounds_and_deltas['num_post_money_valuation_7']*1e6
df_funding_rounds_and_deltas['num_post_money_valuation_8'] = df_funding_rounds_and_deltas['num_post_money_valuation_8']*1e6
df_funding_rounds_and_deltas['num_post_money_valuation_9'] = df_funding_rounds_and_deltas['num_post_money_valuation_9']*1e6
df_funding_rounds_and_deltas['num_post_money_valuation_10'] = df_funding_rounds_and_deltas['num_post_money_valuation_10']*1e6
df_funding_rounds_and_deltas['num_post_money_valuation_11'] = df_funding_rounds_and_deltas['num_post_money_valuation_11']*1e6

create log value funding data

In [ ]:
df_log_total_raised_to_date = df_funding_rounds_and_deltas[["company_name","total_raised_to_date"]].fillna(0)
df_log_total_raised_to_date = df_log_total_raised_to_date[(df_log_total_raised_to_date['total_raised_to_date'] != 0)]
df_log_total_raised_to_date['log10_total_raised_to_date'] = np.log10(df_log_total_raised_to_date['total_raised_to_date'])
df_log_total_raised_to_date.drop(columns=['total_raised_to_date'], inplace = True)

df_log_max_post_money_valuation = df_funding_rounds_and_deltas[["company_name","max_post_money_valuation"]].fillna(0)
df_log_max_post_money_valuation = df_log_max_post_money_valuation[(df_log_max_post_money_valuation['max_post_money_valuation'] != 0)]
df_log_max_post_money_valuation['log10_max_post_money_valuation'] = np.log10(df_log_max_post_money_valuation['max_post_money_valuation'])
df_log_max_post_money_valuation.drop(columns=['max_post_money_valuation'], inplace = True)

df_log_max_post_money_valuation

create new_to_followon_investor_ratio

In [ ]:
df_new_to_followon_investor_ratio = df_funding_rounds_and_deltas[["company_name","num_sum_investors"]].merge(df_funding_rounds_and_deltas[["company_name","num_sum_follow_on_investors"]], on='company_name', how='left')
df_new_to_followon_investor_ratio['new_to_followon_investor_ratio'] = df_new_to_followon_investor_ratio['num_sum_investors']/df_new_to_followon_investor_ratio['num_sum_follow_on_investors']
df_new_to_followon_investor_ratio = df_new_to_followon_investor_ratio.replace([np.inf, -np.inf], np.nan)
df_new_to_followon_investor_ratio.dropna(inplace = True)
df_new_to_followon_investor_ratio.drop(columns=['num_sum_investors', 'num_sum_follow_on_investors'], inplace = True)
print(df_new_to_followon_investor_ratio.shape)

Company Age

In [ ]:
time = datetime.datetime.today()
print(time)
print(type(time))
df_company_age = df_funding_rounds_and_deltas[["company_name","founded_date"]]
df_company_age["company_age"] = (time - df_round_dates['founded_date']).dt.days
df_company_age.head()

Save deltas and company_age

In [ ]:
with pd.ExcelWriter(base_path + 'Analysis/Round_deltas and company age.xlsx') as writer:
    df_difference_between_dates.to_excel(writer, sheet_name= "Round Deltas", index=False)
    df_funding_rounds_and_deltas.to_excel(writer, sheet_name= "Round Deltas All Info", index=False)
    df_company_age.to_excel(writer, sheet_name= "Company Age", index=False)
    
print("File save to " + base_path + 'Analysis/Round_deltas.xlsx ' + str(os.path.isfile(base_path + 'Analysis/Round_deltas.xlsx')) + " File check")

# Create Company Names (Independent Variable)
Different Names which startups had over time Name1, Name2, etc.

In [ ]:
df_company_names_in_versions = pd.read_excel(base_path + "Analysis/Company name changes and current.xlsx", skiprows=0)
df_company_names_in_versions.drop(['twitter_id'], axis=1, inplace = True)
df_company_names_in_versions = df_company_names_in_versions.rename(index=str, columns={"current_name" : "company_name"})
df_company_names_in_versions.head()

In [ ]:
#df_company_names_in_versions["company_name"].value_counts()

# Create Company Twitter ID (Independent Variable)
Different Twitter Handles which startups had over time twitter_id_0, twitter_id_1, etc.

This can be merged on the twitter_id column with df_company_names_in_versions

In [ ]:
df_twitter_ids_in_versions = pd.read_excel(base_path + "Analysis/Twitter_id represented in Versions.xlsx","df_twitter_id_VX", skiprows=0)
df_twitter_ids_in_versions["has_twitter"] = True
df_twitter_ids_in_versions.head()

In [ ]:
#df_twitter_ids_in_versions.loc[df_twitter_ids_in_versions['company_name'] == "sweetspot"]
#df_twitter_ids_in_versions["company_name"].value_counts()

# Create Company Twitter Handles (Independent Variable)
Different Twitter Handles which startups had over time twitter_handle_0, twitter_handle_1, etc.

This can be merged on the twitter_id column with df_company_names_in_versions

In [ ]:
df_twitter_handles_in_versions = pd.read_excel(base_path + "Analysis/Twitter handle changes and current.xlsx", skiprows=0)
df_twitter_handles_in_versions = df_twitter_handles_in_versions[["twitter_id","current_twitter_handle","twitter_handle_0","twitter_handle_1"]]
df_twitter_handles_in_versions = df_twitter_handles_in_versions.rename(index=str, columns={"twitter_id" : "current_twitter_id"})
df_twitter_handles_in_versions.head()

In [ ]:
#df_twitter_ids_in_versions.loc[df_twitter_ids_in_versions['company_name'] == "sweetspot"]
#df_twitter_ids_in_versions["company_name"].value_counts()

# Create Startup Websites (Independent Variable)

In [ ]:
df_company_websites = pd.read_excel(base_path + "Pitchbook_Crunchbase_Tracxn_Raw/Don't touch " + version_id + "/" + version_id + ".xlsx", version_id + "website_url_scrape", skiprows=0)
df_company_websites_no_duplicates_no_nan = df_company_websites[df_company_websites.website_url.notnull()].drop_duplicates()
df_company_websites_no_duplicates_no_nan["has_website"] = True
df_company_websites_no_duplicates_no_nan.head()

In [ ]:
#show duplicate startups with different websites
#should be empty!
df_company_duplicates = df_company_websites["company_name"]
df_company_duplicates = df_company_websites[df_company_duplicates.isin(df_company_duplicates[df_company_duplicates.duplicated()])].sort_values("company_name")
df_company_duplicates.head()

In [ ]:
accessible_websites = [] # to be merged with df_company_websites
scraped_websites_folder = os.path.join(base_path + "/outputs/Scraped Websites")

for root, dirs, files in os.walk(scraped_websites_folder):
    print(root, dirs, files)
    for file in files:
        if file.endswith(".json"):
            #print(os.path.splitext(file)[0])
            accessible_websites.append(os.path.splitext(file)[0])
            
print(accessible_websites)

df_accessible_websites = pd.DataFrame()
df_accessible_websites["company_name"] = accessible_websites
df_accessible_websites["has_accessable_website"] = True
df_accessible_websites.head()

In [ ]:
df_company_websites_and_accessability = df_company_websites.merge(df_accessible_websites, on='company_name', how='left')
df_company_websites_and_accessability.head()

In [ ]:
# Remember to fill companies which don't have a website with nan based on whether a website is reachable. get this info from the folder where all websites are stored.

# Create Tweet Count, Tweet Frequency and Twitter Account Age (Independent Variable)

In [ ]:
df_company_tweet_stats = pd.read_csv(filepath_or_buffer = "/content/drive/My Drive/USERNAME/Analysis/Company tweet stats.csv")
df_company_tweet_stats.head()

# Create Network Relevance Dataframes (Independent Variable)

In [ ]:
df_followers_sum_all_users = pd.read_excel(base_path + "Analysis/Network Relevance/Summaries/df_followers_sum_all_users.xlsx", 'VX', skiprows=0)
df_followers_total_matched_words = pd.read_excel(base_path + "Analysis/Network Relevance/Summaries/df_followers_total_matched_words.xlsx", 'VX', skiprows=0)

df_friends_sum_all_users = pd.read_excel(base_path + "Analysis/Network Relevance/Summaries/df_friends_sum_all_users.xlsx", 'VX', skiprows=0)
df_friends_total_matched_words = pd.read_excel(base_path + "Analysis/Network Relevance/Summaries/df_friends_total_matched_words.xlsx", 'VX', skiprows=0)

df_fnf_sum_all_users = pd.read_excel(base_path + "Analysis/Network Relevance/Summaries/df_fnf_sum_all_users.xlsx", 'VX', skiprows=0)
df_fnf_total_matched_words = pd.read_excel(base_path + "Analysis/Network Relevance/Summaries/df_fnf_total_matched_words.xlsx", 'VX', skiprows=0)

# RENAME:
df_followers_sum_all_users = df_followers_sum_all_users.rename(index=str, columns={"V1_followers_match_count":"followers_match_count_V01", "V2_followers_match_count":"followers_match_count_V02", "V3_followers_match_count":"followers_match_count_V03", "V4_followers_match_count":"followers_match_count_V04", "V5_followers_match_count":"followers_match_count_V05", "V6_followers_match_count":"followers_match_count_V06", "V7_followers_match_count":"followers_match_count_V07", "V8_followers_match_count":"followers_match_count_V08", "V9_followers_match_count":"followers_match_count_V09", "V10_followers_match_count":"followers_match_count_V10", "V11_followers_match_count":"followers_match_count_V11", "V12_followers_match_count":"followers_match_count_V12", "V13_followers_match_count":"followers_match_count_V13", "V14_followers_match_count":"followers_match_count_V14"})
df_followers_total_matched_words = df_followers_total_matched_words.rename(index=str, columns={"V1_followers_match_w_count":"followers_match_w_count_V01", "V2_followers_match_w_count":"followers_match_w_count_V02", "V3_followers_match_w_count":"followers_match_w_count_V03", "V4_followers_match_w_count":"followers_match_w_count_V04", "V5_followers_match_w_count":"followers_match_w_count_V05", "V6_followers_match_w_count":"followers_match_w_count_V06", "V7_followers_match_w_count":"followers_match_w_count_V07", "V8_followers_match_w_count":"followers_match_w_count_V08", "V9_followers_match_w_count":"followers_match_w_count_V09", "V10_followers_match_w_count":"followers_match_w_count_V10", "V11_followers_match_w_count":"followers_match_w_count_V11", "V12_followers_match_w_count":"followers_match_w_count_V12", "V13_followers_match_w_count":"followers_match_w_count_V13", "V14_followers_match_w_count":"followers_match_w_count_V14"})

df_friends_sum_all_users = df_friends_sum_all_users.rename(index=str, columns={"V1_friends_match_count":"friends_match_count_V01", "V2_friends_match_count":"friends_match_count_V02", "V3_friends_match_count":"friends_match_count_V03", "V4_friends_match_count":"friends_match_count_V04", "V5_friends_match_count":"friends_match_count_V05", "V6_friends_match_count":"friends_match_count_V06", "V7_friends_match_count":"friends_match_count_V07", "V8_friends_match_count":"friends_match_count_V08", "V9_friends_match_count":"friends_match_count_V09", "V10_friends_match_count":"friends_match_count_V10", "V11_friends_match_count":"friends_match_count_V11", "V12_friends_match_count":"friends_match_count_V12", "V13_friends_match_count":"friends_match_count_V13", "V14_friends_match_count":"friends_match_count_V14"})
df_friends_total_matched_words = df_friends_total_matched_words.rename(index=str, columns={"V1_friends_match_w_count":"friends_match_w_count_V01", "V2_friends_match_w_count":"friends_match_w_count_V02", "V3_friends_match_w_count":"friends_match_w_count_V03", "V4_friends_match_w_count":"friends_match_w_count_V04", "V5_friends_match_w_count":"friends_match_w_count_V05", "V6_friends_match_w_count":"friends_match_w_count_V06", "V7_friends_match_w_count":"friends_match_w_count_V07", "V8_friends_match_w_count":"friends_match_w_count_V08", "V9_friends_match_w_count":"friends_match_w_count_V09", "V10_friends_match_w_count":"friends_match_w_count_V10", "V11_friends_match_w_count":"friends_match_w_count_V11", "V12_friends_match_w_count":"friends_match_w_count_V12", "V13_friends_match_w_count":"friends_match_w_count_V13", "V14_friends_match_w_count":"friends_match_w_count_V14"})

df_fnf_sum_all_users = df_fnf_sum_all_users.rename(index=str, columns={"V1_fnf_match_count":"unweighted_bi_direct_match_count_V01", "V2_fnf_match_count":"unweighted_bi_direct_match_count_V02", "V3_fnf_match_count":"unweighted_bi_direct_match_count_V03", "V4_fnf_match_count":"unweighted_bi_direct_match_count_V04", "V5_fnf_match_count":"unweighted_bi_direct_match_count_V05", "V6_fnf_match_count":"unweighted_bi_direct_match_count_V06", "V7_fnf_match_count":"unweighted_bi_direct_match_count_V07", "V8_fnf_match_count":"unweighted_bi_direct_match_count_V08", "V9_fnf_match_count":"unweighted_bi_direct_match_count_V09", "V10_fnf_match_count":"unweighted_bi_direct_match_count_V10", "V11_fnf_match_count":"unweighted_bi_direct_match_count_V11", "V12_fnf_match_count":"unweighted_bi_direct_match_count_V12", "V13_fnf_match_count":"unweighted_bi_direct_match_count_V13", "V14_fnf_match_count":"unweighted_bi_direct_match_count_V14"})
df_fnf_total_matched_words = df_fnf_total_matched_words.rename(index=str, columns={"V1_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V01", "V2_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V02", "V3_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V03", "V4_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V04", "V5_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V05", "V6_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V06", "V7_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V07", "V8_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V08", "V9_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V09", "V10_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V10", "V11_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V11", "V12_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V12", "V13_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V13", "V14_fnf_match_w_count":"unweighted_bi_direct_match_w_count_V14"})

df_followers_sum_all_users.head()

Sum all matched users

In [ ]:
df_followers_sum_all_users["followers_match_count_mean"] = df_followers_sum_all_users[[h for h in list(df_followers_sum_all_users) if "V" in h]].mean(axis=1)
df_followers_sum_all_users["followers_match_count_var"] = df_followers_sum_all_users[[h for h in list(df_followers_sum_all_users) if "V" in h]].var(axis=1)
df_followers_sum_all_users["followers_match_count_std"] = df_followers_sum_all_users[[h for h in list(df_followers_sum_all_users) if "V" in h]].std(axis=1)
df_followers_sum_all_users_temp = df_followers_sum_all_users[[h for h in list(df_followers_sum_all_users) if "match_count_V" in h]]
df_followers_sum_all_users_temp.reset_index(drop=True, inplace = True)
df_followers_sum_all_users["followers_match_count_delta_survey"] = delta_survey(df_followers_sum_all_users_temp)
df_followers_sum_all_users["followers_match_count_rel_difference"] = rel_diff_survey(df_followers_sum_all_users_temp)

df_friends_sum_all_users["friends_match_count_mean"] = df_friends_sum_all_users[[h for h in list(df_friends_sum_all_users) if "V" in h]].mean(axis=1)
df_friends_sum_all_users["friends_match_count_var"] = df_friends_sum_all_users[[h for h in list(df_friends_sum_all_users) if "V" in h]].var(axis=1)
df_friends_sum_all_users["friends_match_count_std"] = df_friends_sum_all_users[[h for h in list(df_friends_sum_all_users) if "V" in h]].std(axis=1)
df_friends_sum_all_users_temp = df_friends_sum_all_users[[h for h in list(df_friends_sum_all_users) if "match_count_V" in h]]
df_friends_sum_all_users_temp.reset_index(drop=True, inplace = True)
df_friends_sum_all_users["friends_match_count_delta_survey"] = delta_survey(df_friends_sum_all_users_temp)
df_friends_sum_all_users["friends_match_count_rel_difference"] = rel_diff_survey(df_friends_sum_all_users_temp)

df_fnf_sum_all_users["unweighted_bi_direct_match_count_mean"] = df_fnf_sum_all_users[[h for h in list(df_fnf_sum_all_users) if "V" in h]].mean(axis=1)
df_fnf_sum_all_users["unweighted_bi_direct_match_count_var"] = df_fnf_sum_all_users[[h for h in list(df_fnf_sum_all_users) if "V" in h]].var(axis=1)
df_fnf_sum_all_users["unweighted_bi_direct_match_count_std"] = df_fnf_sum_all_users[[h for h in list(df_fnf_sum_all_users) if "V" in h]].std(axis=1)
df_fnf_sum_all_users_temp = df_fnf_sum_all_users[[h for h in list(df_fnf_sum_all_users) if "match_count_V" in h]]
df_fnf_sum_all_users_temp.reset_index(drop=True, inplace = True)
df_fnf_sum_all_users["unweighted_bi_direct_match_count_delta_survey"] = delta_survey(df_fnf_sum_all_users_temp)
df_fnf_sum_all_users["unweighted_bi_direct_match_count_rel_difference"] = rel_diff_survey(df_fnf_sum_all_users_temp)

df_followers_sum_all_users.head()

Sum all matched words

In [ ]:
df_followers_total_matched_words["followers_match_w_count_mean"] = df_followers_total_matched_words[[h for h in list(df_followers_total_matched_words) if "V" in h]].mean(axis=1)
df_followers_total_matched_words["followers_match_w_count_var"] = df_followers_total_matched_words[[h for h in list(df_followers_total_matched_words) if "V" in h]].var(axis=1)
df_followers_total_matched_words["followers_match_w_count_std"] = df_followers_total_matched_words[[h for h in list(df_followers_total_matched_words) if "V" in h]].std(axis=1)
df_followers_total_matched_words["followers_match_w_count_per_user_avg"] = df_followers_total_matched_words['followers_match_w_count_mean']/df_followers_sum_all_users['followers_match_count_mean']
df_followers_total_matched_words_temp = df_followers_total_matched_words[[h for h in list(df_followers_total_matched_words) if "match_w_count_V" in h]]
df_followers_total_matched_words_temp.reset_index(drop=True, inplace = True)
df_followers_total_matched_words["followers_match_w_count_delta_survey"] = delta_survey(df_followers_total_matched_words_temp)
df_followers_total_matched_words["followers_match_w_count_rel_difference"] = rel_diff_survey(df_followers_total_matched_words_temp)

df_friends_total_matched_words["friends_match_w_count_mean"] = df_friends_total_matched_words[[h for h in list(df_friends_total_matched_words) if "V" in h]].mean(axis=1)
df_friends_total_matched_words["friends_match_w_count_var"] = df_friends_total_matched_words[[h for h in list(df_friends_total_matched_words) if "V" in h]].var(axis=1)
df_friends_total_matched_words["friends_match_w_count_std"] = df_friends_total_matched_words[[h for h in list(df_friends_total_matched_words) if "V" in h]].std(axis=1)
df_friends_total_matched_words["friends_match_w_count_per_user_avg"] = df_friends_total_matched_words["friends_match_w_count_mean"]/df_friends_sum_all_users["friends_match_count_mean"]
df_friends_total_matched_words_temp = df_friends_total_matched_words[[h for h in list(df_friends_total_matched_words) if "match_w_count_V" in h]]
df_friends_total_matched_words_temp.reset_index(drop=True, inplace = True)
df_friends_total_matched_words["friends_match_w_count_delta_survey"] = delta_survey(df_friends_total_matched_words_temp)
df_friends_total_matched_words["friends_match_w_count_rel_difference"] = rel_diff_survey(df_friends_total_matched_words_temp)

df_fnf_total_matched_words["unweighted_bi_direct_match_w_count_mean"] = df_fnf_total_matched_words[[h for h in list(df_fnf_total_matched_words) if "V" in h]].mean(axis=1)
df_fnf_total_matched_words["unweighted_bi_direct_match_w_count_var"] = df_fnf_total_matched_words[[h for h in list(df_fnf_total_matched_words) if "V" in h]].var(axis=1)
df_fnf_total_matched_words["unweighted_bi_direct_match_w_count_std"] = df_fnf_total_matched_words[[h for h in list(df_fnf_total_matched_words) if "V" in h]].std(axis=1)
df_fnf_total_matched_words["unweighted_bi_direct_match_w_per_user_count_avg"] = df_fnf_total_matched_words["unweighted_bi_direct_match_w_count_mean"]/df_fnf_sum_all_users["unweighted_bi_direct_match_count_mean"]
df_fnf_total_matched_words_temp = df_fnf_total_matched_words[[h for h in list(df_fnf_total_matched_words) if "match_w_count_V" in h]]
df_fnf_total_matched_words_temp.reset_index(drop=True, inplace = True)
df_fnf_total_matched_words["unweighted_bi_direct_match_w_count_delta_survey"] = delta_survey(df_fnf_total_matched_words_temp)
df_fnf_total_matched_words["unweighted_bi_direct_match_w_count_rel_difference"] = rel_diff_survey(df_fnf_total_matched_words_temp)

df_followers_total_matched_words.head()

# Create Accelerated Dataframe (Independent Variable)

In [ ]:
df_connections = pd.read_excel(base_path + "Analysis/SNA/company_investor_edgelist_attributes.xlsx", 'Connections', header=None, skiprows=0, names=['company_name','accelerated'], usecols="A,C")
print("df_connections", df_connections.shape)
df_accelerated = df_connections[df_connections['accelerated'].isin(["Accelerated", "Accelerated and Invested"])]
print("df_accelerated", df_accelerated.shape)
df_accelerated.drop_duplicates(subset="company_name", keep='first', inplace=True)
print("df_accelerated_dupl_drop", df_accelerated.shape)
df_accelerated = df_accelerated.reset_index(drop=True)
df_accelerated['accelerated'].replace({'Accelerated and Invested': True,'Accelerated': True},inplace = True)
df_accelerated.tail()

In [ ]:
df_accelerated.loc[df_accelerated['company_name'] == "reinfer"]
#df_accelerated.loc[df_accelerated['accelerated'] == "Accelerated"]
#df_accelerated["company_name"].value_counts()

# Create Invested Dataframe (Independent Variable)

In [ ]:
df_connections = pd.read_excel(base_path + "Analysis/SNA/company_investor_edgelist_attributes.xlsx", 'Connections', header=None, skiprows=0, names=['company_name','invested'], usecols="A,C")
df_invested = df_connections[df_connections['invested'].isin(["Invested", "Accelerated and Invested"])]
df_invested.drop_duplicates(subset=None, keep='first', inplace=True)
df_invested = df_invested.reset_index(drop=True)
df_invested['invested'].replace({'Accelerated and Invested': True},inplace = True)
df_invested['invested'].replace({'Invested': True},inplace = True)
df_invested.drop_duplicates(inplace=True)
df_invested.head()

In [ ]:
df_invested.loc[df_invested['company_name'] == "reinfer"]

# Create Description Dataframe (Independent Variable)

In [ ]:
df_before_split = pd.read_excel(base_path + "Pitchbook_Crunchbase_Tracxn_Raw/Don't touch " + version_id + "/" + version_id + ".xlsx" , 'V14df_all', header=None, skiprows=1, names=['company_name','desc','source'], usecols="A,B,R")
df_before_split.head()

D = df_before_split.to_dict("index")
print(D)
Desc_dict = {}
# i = 0
for key, row_dict in D.items():
#   i+=1
#   if i > 10:
#     break
#   print(key, row_dict)
  c_name = row_dict["company_name"]
  if c_name not in Desc_dict:
    Desc_dict[c_name] = {}
  source = row_dict["source"]
  desc = row_dict["desc"]
  Desc_dict[c_name][source]=desc

#pp.pprint(Desc_dict)

In [ ]:
desc_df = pd.DataFrame.from_dict(Desc_dict, orient='index')
desc_df.reset_index(level=0, inplace=True)
desc_df = desc_df.rename(index=str, columns={"index" : "company_name", "Pitchbook" : "desc_pitchbook", "Tracxn" : "desc_tracxn", "Crunchbase" : "desc_crunchbase"})
desc_df.head()

In [ ]:
src_pitchbook_df = desc_df[["company_name","desc_pitchbook"]]
src_pitchbook_df = src_pitchbook_df[src_pitchbook_df.desc_pitchbook.notnull()]
src_pitchbook_df["src_pitchbook"] = True
src_pitchbook_df.head()

In [ ]:
src_crunchbase_df = desc_df[["company_name","desc_crunchbase"]]
src_crunchbase_df = src_crunchbase_df[src_crunchbase_df.desc_crunchbase.notnull()]
src_crunchbase_df["src_crunchbase"] = True
src_crunchbase_df.head()

In [ ]:
src_tracxn_df = desc_df[["company_name","desc_tracxn"]]
src_tracxn_df = src_tracxn_df[src_tracxn_df.desc_tracxn.notnull()]
src_tracxn_df["src_tracxn"] = True
src_tracxn_df.head()

# Webrank import

In [ ]:
df_webrank = pd.read_excel(base_path + "Analysis/FILEPATH" , 'df_webrank_no_dupl_no_nan', skiprows=0, usecols="A,B,C")
print(df_webrank.shape)
df_webrank.head()

In [ ]:
df_webrank.loc[df_webrank['company_name'] == "petagene"]

# Create company B2B or B2C Dataframe (Independent Variable)

In [ ]:
df_b2b = pd.read_excel(base_path + "Analysis/Industry Match Manual.xlsx" , 'description', header=None, skiprows=1, names=['company_name','b2b'], usecols="A,B")
df_b2b = df_b2b.drop_duplicates(subset='company_name', keep="first")
df_b2b.head()

# Create company HQ city Dataframe (Independent Variable)

In [ ]:
df_hq_city = pd.read_excel(base_path + "Analysis/companies HQ city.xlsx" , 'df_HQ_city', header=None, skiprows=1, names=['company_name','hq_city'], usecols="A,B")
df_hq_city = df_hq_city.drop_duplicates(subset='company_name', keep="first")
df_hq_city.head()

# Create Twitter SNA Dataframe (Independent Variable)

In [ ]:
sna_in_versions_file = base_path + "Analysis/SNA/SNA in Versions.xlsx"

# import all dataframes with all columns
df_startupsfnf_all_columns = pd.read_excel(sna_in_versions_file, 'startupsfnf_all_columns', skiprows=0).drop_duplicates(subset='company_name', keep="first").reset_index(drop=True)
df_startupsfriends_all_columns = pd.read_excel(sna_in_versions_file, 'startupsfriends_all_columns', skiprows=0).drop_duplicates(subset='company_name', keep="first").reset_index(drop=True)
df_startupsfollowers_all_columns = pd.read_excel(sna_in_versions_file, 'startupsfollower_all_columns', skiprows=0).drop_duplicates(subset='company_name', keep="first").reset_index(drop=True)
df_unweighted_bi_direct_all_columns = pd.read_excel(sna_in_versions_file, 'bi_direct_all_columns', skiprows=0).drop_duplicates(subset='company_name', keep="first").reset_index(drop=True)
df_weighted_bi_direct_all_columns = pd.read_excel(sna_in_versions_file, 'weighted_bi_direct_all_columns', skiprows=0).drop_duplicates(subset='company_name', keep="first").reset_index(drop=True)


# drop V0, info headers, and twitter_ids from all dfs
info_header = ['event', 'name_change', 'company_name_current', 'dropper']
df_startupsfnf_all_columns.drop([c for c in list(df_startupsfnf_all_columns) if "V0" in c or "twitter" in c] + info_header, axis=1, inplace = True)
df_startupsfriends_all_columns.drop([c for c in list(df_startupsfriends_all_columns) if "V0" in c or "twitter" in c] + info_header, axis=1, inplace = True)
df_startupsfollowers_all_columns.drop([c for c in list(df_startupsfollowers_all_columns) if "V0" in c or "twitter" in c] + info_header, axis=1, inplace = True)
df_unweighted_bi_direct_all_columns.drop([c for c in list(df_unweighted_bi_direct_all_columns) if "V0" in c or "twitter" in c] + info_header, axis=1, inplace = True)
df_weighted_bi_direct_all_columns.drop([c for c in list(df_weighted_bi_direct_all_columns) if "V0" in c or "twitter" in c] + info_header, axis=1, inplace = True)

# rename columns
df_startupsfnf_all_columns = df_startupsfnf_all_columns.rename(index=str, columns={"degree_V1":"fnf_degree_V01", "degree_centrality_V1":"fnf_degree_centrality_V01", "degree_V2":"fnf_degree_V02", "degree_centrality_V2":"fnf_degree_centrality_V02", "degree_V3":"fnf_degree_V03", "degree_centrality_V3":"fnf_degree_centrality_V03", "degree_V4":"fnf_degree_V04", "degree_centrality_V4":"fnf_degree_centrality_V04", "degree_V5":"fnf_degree_V05", "degree_centrality_V5":"fnf_degree_centrality_V05", "degree_V6" : "fnf_degree_V06", "degree_centrality_V6":"fnf_degree_centrality_V06", "degree_V7":"fnf_degree_V07", "degree_centrality_V7":"fnf_degree_centrality_V07", "degree_V8":"fnf_degree_V08", "degree_centrality_V8":"fnf_degree_centrality_V08", "degree_V9":"fnf_degree_V09", "degree_centrality_V9":"fnf_degree_centrality_V09", "degree_V10":"fnf_degree_V10", "degree_centrality_V10":"fnf_degree_centrality_V10", "degree_V11":"fnf_degree_V11", "degree_centrality_V11":"fnf_degree_centrality_V11", "degree_V12":"fnf_degree_V12", "degree_centrality_V12":"fnf_degree_centrality_V12", "degree_V13":"fnf_degree_V13", "degree_centrality_V13":"fnf_degree_centrality_V13", "degree_V14":"fnf_degree_V14", "degree_centrality_V14":"fnf_degree_centrality_V14"})
df_startupsfriends_all_columns = df_startupsfriends_all_columns.rename(index=str, columns={"out_degree_V1":"friends_degree_V01", "degree_centrality_V1":"friends_degree_centrality_V01", "out_degree_V2":"friends_degree_V02", "degree_centrality_V2":"friends_degree_centrality_V02", "out_degree_V3":"friends_degree_V03", "degree_centrality_V3":"friends_degree_centrality_V03", "out_degree_V4":"friends_degree_V04", "degree_centrality_V4":"friends_degree_centrality_V04", "out_degree_V5":"friends_degree_V05", "degree_centrality_V5":"friends_degree_centrality_V05", "out_degree_V6" : "friends_degree_V06", "degree_centrality_V6":"friends_degree_centrality_V06", "out_degree_V7":"friends_degree_V07", "degree_centrality_V7":"friends_degree_centrality_V07", "out_degree_V8":"friends_degree_V08", "degree_centrality_V8":"friends_degree_centrality_V08", "out_degree_V9":"friends_degree_V09", "degree_centrality_V9":"friends_degree_centrality_V09", "out_degree_V10":"friends_degree_V10", "degree_centrality_V10":"friends_degree_centrality_V10", "out_degree_V11":"friends_degree_V11", "degree_centrality_V11":"friends_degree_centrality_V11", "out_degree_V12":"friends_degree_V12", "degree_centrality_V12":"friends_degree_centrality_V12", "out_degree_V13":"friends_degree_V13", "degree_centrality_V13":"friends_degree_centrality_V13", "out_degree_V14":"friends_degree_V14", "degree_centrality_V14":"friends_degree_centrality_V14"})
df_startupsfollowers_all_columns = df_startupsfollowers_all_columns.rename(index=str, columns={"in_degree_V1":"followers_degree_V01", "degree_centrality_V1":"followers_degree_centrality_V01", "in_degree_V2":"followers_degree_V02", "degree_centrality_V2":"followers_degree_centrality_V02", "in_degree_V3":"followers_degree_V03", "degree_centrality_V3":"followers_degree_centrality_V03", "in_degree_V4":"followers_degree_V04", "degree_centrality_V4":"followers_degree_centrality_V04", "in_degree_V5":"followers_degree_V05", "degree_centrality_V5":"followers_degree_centrality_V05", "in_degree_V6" : "followers_degree_V06", "degree_centrality_V6":"followers_degree_centrality_V06", "in_degree_V7":"followers_degree_V07", "degree_centrality_V7":"followers_degree_centrality_V07", "in_degree_V8":"followers_degree_V08", "degree_centrality_V8":"followers_degree_centrality_V08", "in_degree_V9":"followers_degree_V09", "degree_centrality_V9":"followers_degree_centrality_V09", "in_degree_V10":"followers_degree_V10", "degree_centrality_V10":"followers_degree_centrality_V10", "in_degree_V11":"followers_degree_V11", "degree_centrality_V11":"followers_degree_centrality_V11", "in_degree_V12":"followers_degree_V12", "degree_centrality_V12":"followers_degree_centrality_V12", "in_degree_V13":"followers_degree_V13", "degree_centrality_V13":"followers_degree_centrality_V13", "in_degree_V14":"followers_degree_V14", "degree_centrality_V14":"followers_degree_centrality_V14"})
df_unweighted_bi_direct_all_columns = df_unweighted_bi_direct_all_columns.rename(index=str, columns={"degree_V1":"unweighted_bi_direct_degree_V01", "degree_centrality_V1":"unweighted_bi_direct_degree_centrality_V01", "degree_V2":"unweighted_bi_direct_degree_V02", "degree_centrality_V2":"unweighted_bi_direct_degree_centrality_V02", "degree_V3":"unweighted_bi_direct_degree_V03", "degree_centrality_V3":"unweighted_bi_direct_degree_centrality_V03", "degree_V4":"unweighted_bi_direct_degree_V04", "degree_centrality_V4":"unweighted_bi_direct_degree_centrality_V04", "degree_V5":"unweighted_bi_direct_degree_V05", "degree_centrality_V5":"unweighted_bi_direct_degree_centrality_V05", "degree_V6" : "unweighted_bi_direct_degree_V06", "degree_centrality_V6":"unweighted_bi_direct_degree_centrality_V06", "degree_V7":"unweighted_bi_direct_degree_V07", "degree_centrality_V7":"unweighted_bi_direct_degree_centrality_V07", "degree_V8":"unweighted_bi_direct_degree_V08", "degree_centrality_V8":"unweighted_bi_direct_degree_centrality_V08", "degree_V9":"unweighted_bi_direct_degree_V09", "degree_centrality_V9":"unweighted_bi_direct_degree_centrality_V09", "degree_V10":"unweighted_bi_direct_degree_V10", "degree_centrality_V10":"unweighted_bi_direct_degree_centrality_V10", "degree_V11":"unweighted_bi_direct_degree_V11", "degree_centrality_V11":"unweighted_bi_direct_degree_centrality_V11", "degree_V12":"unweighted_bi_direct_degree_V12", "degree_centrality_V12":"unweighted_bi_direct_degree_centrality_V12", "degree_V13":"unweighted_bi_direct_degree_V13", "degree_centrality_V13":"unweighted_bi_direct_degree_centrality_V13", "degree_V14":"unweighted_bi_direct_degree_V14", "degree_centrality_V14":"unweighted_bi_direct_degree_centrality_V14"})
df_weighted_bi_direct_all_columns = df_weighted_bi_direct_all_columns.rename(index=str, columns={"weighted_degree_V1":"fr_weighted_bi_direct_degree_V01", "weighted_degree_centrality_V1":"fr_weighted_bi_direct_degree_centrality_V01", "weighted_degree_V2":"fr_weighted_bi_direct_degree_V02", "weighted_degree_centrality_V2":"fr_weighted_bi_direct_degree_centrality_V02", "weighted_degree_V3":"fr_weighted_bi_direct_degree_V03", "weighted_degree_centrality_V3":"fr_weighted_bi_direct_degree_centrality_V03", "weighted_degree_V4":"fr_weighted_bi_direct_degree_V04", "weighted_degree_centrality_V4":"fr_weighted_bi_direct_degree_centrality_V04", "weighted_degree_V5":"fr_weighted_bi_direct_degree_V05", "weighted_degree_centrality_V5":"fr_weighted_bi_direct_degree_centrality_V05", "weighted_degree_V6" : "fr_weighted_bi_direct_degree_V06", "weighted_degree_centrality_V6":"fr_weighted_bi_direct_degree_centrality_V06", "weighted_degree_V7":"fr_weighted_bi_direct_degree_V07", "weighted_degree_centrality_V7":"fr_weighted_bi_direct_degree_centrality_V07", "weighted_degree_V8":"fr_weighted_bi_direct_degree_V08", "weighted_degree_centrality_V8":"fr_weighted_bi_direct_degree_centrality_V08", "weighted_degree_V9":"fr_weighted_bi_direct_degree_V09", "weighted_degree_centrality_V9":"fr_weighted_bi_direct_degree_centrality_V09", "weighted_degree_V10":"fr_weighted_bi_direct_degree_V10", "weighted_degree_centrality_V10":"fr_weighted_bi_direct_degree_centrality_V10", "weighted_degree_V11":"fr_weighted_bi_direct_degree_V11", "weighted_degree_centrality_V11":"fr_weighted_bi_direct_degree_centrality_V11", "weighted_degree_V12":"fr_weighted_bi_direct_degree_V12", "weighted_degree_centrality_V12":"fr_weighted_bi_direct_degree_centrality_V12", "weighted_degree_V13":"fr_weighted_bi_direct_degree_V13", "weighted_degree_centrality_V13":"fr_weighted_bi_direct_degree_centrality_V13", "weighted_degree_V14":"fr_weighted_bi_direct_degree_V14", "weighted_degree_centrality_V14":"fr_weighted_bi_direct_degree_centrality_V14"})

# reduce initial column to only degree dataframe
df_startupsfnf_degree = df_startupsfnf_all_columns[[c for c in list(df_startupsfnf_all_columns) if "centrality" not in c ]]#df_startupsfnf_degree = df_startupsfnf_all_columns[[c for c in list(df_startupsfnf_all_columns) if "centrality" not in c]]
df_startupsfriends_degree = df_startupsfriends_all_columns[[c for c in list(df_startupsfriends_all_columns) if "centrality" not in c ]]
df_startupsfollowers_degree = df_startupsfollowers_all_columns[[c for c in list(df_startupsfollowers_all_columns) if "centrality" not in c ]]
df_unweighted_bi_direct_degree = df_unweighted_bi_direct_all_columns[[c for c in list(df_unweighted_bi_direct_all_columns) if "centrality" not in c ]]
df_weighted_bi_direct_degree = df_weighted_bi_direct_all_columns[[c for c in list(df_weighted_bi_direct_all_columns) if "centrality" not in c ]]

# reduce initial column to only degree_centrality dataframe
df_startupsfnf_degree_centrality = df_startupsfnf_all_columns[["company_name"] + [c for c in list(df_startupsfnf_all_columns) if "centrality" in c]]
df_startupsfriends_degree_centrality = df_startupsfriends_all_columns[["company_name"] + [c for c in list(df_startupsfriends_all_columns) if "centrality" in c]]
df_startupsfollowers_degree_centrality = df_startupsfollowers_all_columns[["company_name"] + [c for c in list(df_startupsfollowers_all_columns) if "centrality" in c]]
df_unweighted_bi_direct_degree_centrality = df_unweighted_bi_direct_all_columns[["company_name"] + [c for c in list(df_unweighted_bi_direct_all_columns) if "centrality" in c]]
df_weighted_bi_direct_degree_centrality = df_weighted_bi_direct_all_columns[["company_name"] + [c for c in list(df_weighted_bi_direct_all_columns) if "centrality" in c]]

df_unweighted_bi_direct_degree_centrality.head()

In [ ]:
df_startupsfollowers_degree.head()

In [ ]:
df_startupsfriends_degree.head()

In [ ]:
df_unweighted_bi_direct_degree.head()

In [ ]:
df_weighted_bi_direct_degree.head()

### Degree

In [ ]:
df_startupsfnf_degree["fnf_degree_var"] = df_startupsfnf_degree[[h for h in list(df_startupsfnf_degree) if "V"]].var(axis=1)
df_startupsfnf_degree["fnf_degree_mean"] = df_startupsfnf_degree[[h for h in list(df_startupsfnf_degree) if "V"]].mean(axis=1)
df_startupsfnf_degree["fnf_degree_std"] = df_startupsfnf_degree[[h for h in list(df_startupsfnf_degree) if "V"]].std(axis=1)
df_startupsfnf_degree_temp = df_startupsfnf_degree[[h for h in list(df_startupsfnf_degree) if "fnf_degree_V" in h]]
df_startupsfnf_degree_temp.reset_index(drop=True, inplace = True)
df_startupsfnf_degree["fnf_degree_delta_survey"] = delta_survey(df_startupsfnf_degree_temp)
df_startupsfnf_degree["fnf_degree_rel_difference"] = rel_diff_survey(df_startupsfnf_degree_temp)

df_startupsfriends_degree["friends_degree_mean"] = df_startupsfriends_degree[[h for h in list(df_startupsfriends_degree) if "V" in h]].mean(axis=1)
df_startupsfriends_degree["friends_degree_var"] = df_startupsfriends_degree[[h for h in list(df_startupsfriends_degree) if "V" in h]].var(axis=1)
df_startupsfriends_degree["friends_degree_std"] = df_startupsfriends_degree[[h for h in list(df_startupsfriends_degree) if "V" in h]].std(axis=1)
df_startupsfriends_degree_temp = df_startupsfriends_degree[[h for h in list(df_startupsfriends_degree) if "friends_degree_V" in h]]
df_startupsfriends_degree_temp.reset_index(drop=True, inplace = True)
df_startupsfriends_degree["friends_degree_delta_survey"] = delta_survey(df_startupsfriends_degree_temp)
df_startupsfriends_degree["friends_degree_rel_difference"] = rel_diff_survey(df_startupsfriends_degree_temp)

df_startupsfollowers_degree["followers_degree_mean"] = df_startupsfollowers_degree[[h for h in list(df_startupsfollowers_degree) if "V" in h]].mean(axis=1)
df_startupsfollowers_degree["followers_degree_var"] = df_startupsfollowers_degree[[h for h in list(df_startupsfollowers_degree) if "V" in h]].var(axis=1)
df_startupsfollowers_degree["followers_degree_std"] = df_startupsfollowers_degree[[h for h in list(df_startupsfollowers_degree) if "V" in h]].std(axis=1)
df_startupsfollowers_degree_temp = df_startupsfollowers_degree[[h for h in list(df_startupsfollowers_degree) if "followers_degree_V" in h]]
df_startupsfollowers_degree_temp.reset_index(drop=True, inplace = True)
df_startupsfollowers_degree["followers_degree_delta_survey"] = delta_survey(df_startupsfollowers_degree_temp)
df_startupsfollowers_degree["followers_degree_rel_difference"] = rel_diff_survey(df_startupsfollowers_degree_temp)

df_unweighted_bi_direct_degree["unweighted_bi_direct_degree_mean"] = df_unweighted_bi_direct_degree[[h for h in list(df_unweighted_bi_direct_degree) if "V" in h]].mean(axis=1)
df_unweighted_bi_direct_degree["unweighted_bi_direct_degree_var"] = df_unweighted_bi_direct_degree[[h for h in list(df_unweighted_bi_direct_degree) if "V" in h]].var(axis=1)
df_unweighted_bi_direct_degree["unweighted_bi_direct_degree_std"] = df_unweighted_bi_direct_degree[[h for h in list(df_unweighted_bi_direct_degree) if "V" in h]].std(axis=1)
df_unweighted_bi_direct_degree_temp = df_unweighted_bi_direct_degree[[h for h in list(df_unweighted_bi_direct_degree) if "unweighted_bi_direct_degree_V" in h]]
df_unweighted_bi_direct_degree_temp.reset_index(drop=True, inplace = True)
df_unweighted_bi_direct_degree["unweighted_bi_direct_degree_delta_survey"] = delta_survey(df_unweighted_bi_direct_degree_temp)
df_unweighted_bi_direct_degree["unweighted_bi_direct_degree_rel_difference"] = rel_diff_survey(df_unweighted_bi_direct_degree_temp)

df_weighted_bi_direct_degree["fr_weighted_bi_direct_degree_mean"] = df_weighted_bi_direct_degree[[h for h in list(df_weighted_bi_direct_degree) if "V" in h]].mean(axis=1)
df_weighted_bi_direct_degree["fr_weighted_bi_direct_degree_var"] = df_weighted_bi_direct_degree[[h for h in list(df_weighted_bi_direct_degree) if "V" in h]].var(axis=1)
df_weighted_bi_direct_degree["fr_weighted_bi_direct_degree_std"] = df_weighted_bi_direct_degree[[h for h in list(df_weighted_bi_direct_degree) if "V" in h]].std(axis=1)
df_weighted_bi_direct_degree_temp = df_weighted_bi_direct_degree[[h for h in list(df_weighted_bi_direct_degree) if "fr_weighted_bi_direct_degree_V" in h]]
df_weighted_bi_direct_degree_temp.reset_index(drop=True, inplace = True)
df_weighted_bi_direct_degree["fr_weighted_bi_direct_degree_delta_survey"] = delta_survey(df_weighted_bi_direct_degree_temp)
df_weighted_bi_direct_degree["fr_weighted_bi_direct_degree_rel_difference"] = rel_diff_survey(df_weighted_bi_direct_degree_temp)

### Degree Centrality

In [ ]:
df_startupsfnf_degree_centrality["fnf_degree_centrality_mean"] = df_startupsfnf_degree_centrality[[h for h in list(df_startupsfnf_degree_centrality) if "V" in h]].mean(axis=1)
df_startupsfnf_degree_centrality["fnf_degree_centrality_var"] = df_startupsfnf_degree_centrality[[h for h in list(df_startupsfnf_degree_centrality) if "V" in h]].var(axis=1)
df_startupsfnf_degree_centrality["fnf_degree_centrality_std"] = df_startupsfnf_degree_centrality[[h for h in list(df_startupsfnf_degree_centrality) if "V" in h]].std(axis=1)
df_startupsfnf_degree_centrality_temp = df_startupsfnf_degree_centrality[[h for h in list(df_startupsfnf_degree_centrality) if "fnf_degree_centrality_V" in h]]
df_startupsfnf_degree_centrality_temp.reset_index(drop=True, inplace = True)
df_startupsfnf_degree_centrality["fnf_degree_centrality_delta_survey"] = delta_survey(df_startupsfnf_degree_centrality_temp)
df_startupsfnf_degree_centrality["fnf_degree_centrality_rel_difference"] = rel_diff_survey(df_startupsfnf_degree_centrality_temp)

df_startupsfriends_degree_centrality["friends_degree_centrality_mean"] = df_startupsfriends_degree_centrality[[h for h in list(df_startupsfriends_degree_centrality) if "V" in h]].mean(axis=1)
df_startupsfriends_degree_centrality["friends_degree_centrality_var"] = df_startupsfriends_degree_centrality[[h for h in list(df_startupsfriends_degree_centrality) if "V" in h]].var(axis=1)
df_startupsfriends_degree_centrality["friends_degree_centrality_std"] = df_startupsfriends_degree_centrality[[h for h in list(df_startupsfriends_degree_centrality) if "V" in h]].std(axis=1)
df_startupsfriends_degree_centrality_temp = df_startupsfriends_degree_centrality[[h for h in list(df_startupsfriends_degree_centrality) if "friends_degree_centrality_V" in h]]
df_startupsfriends_degree_centrality_temp.reset_index(drop=True, inplace = True)
df_startupsfriends_degree_centrality["friends_degree_centrality_delta_survey"] = delta_survey(df_startupsfriends_degree_centrality_temp)
df_startupsfriends_degree_centrality["friends_degree_centrality_rel_difference"] = rel_diff_survey(df_startupsfriends_degree_centrality_temp)

df_startupsfollowers_degree_centrality["followers_degree_centrality_mean"] = df_startupsfollowers_degree_centrality[[h for h in list(df_startupsfollowers_degree_centrality) if "V" in h]].mean(axis=1)
df_startupsfollowers_degree_centrality["followers_degree_centrality_var"] = df_startupsfollowers_degree_centrality[[h for h in list(df_startupsfollowers_degree_centrality) if "V" in h]].var(axis=1)
df_startupsfollowers_degree_centrality["followers_degree_centrality_std"] = df_startupsfollowers_degree_centrality[[h for h in list(df_startupsfollowers_degree_centrality) if "V" in h]].std(axis=1)
df_startupsfollowers_degree_centrality_temp = df_startupsfollowers_degree_centrality[[h for h in list(df_startupsfollowers_degree_centrality) if "followers_degree_centrality_V" in h]]
df_startupsfollowers_degree_centrality_temp.reset_index(drop=True, inplace = True)
df_startupsfollowers_degree_centrality["followers_degree_centrality_delta_survey"] = delta_survey(df_startupsfollowers_degree_centrality_temp)
df_startupsfollowers_degree_centrality["followers_degree_centrality_rel_difference"] = rel_diff_survey(df_startupsfollowers_degree_centrality_temp)

df_unweighted_bi_direct_degree_centrality["unweighted_bi_direct_degree_centrality_mean"] = df_unweighted_bi_direct_degree_centrality[[h for h in list(df_unweighted_bi_direct_degree_centrality) if "V" in h]].mean(axis=1)
df_unweighted_bi_direct_degree_centrality["unweighted_bi_direct_degree_centrality_var"] = df_unweighted_bi_direct_degree_centrality[[h for h in list(df_unweighted_bi_direct_degree_centrality) if "V" in h]].var(axis=1)
df_unweighted_bi_direct_degree_centrality["unweighted_bi_direct_degree_centrality_std"] = df_unweighted_bi_direct_degree_centrality[[h for h in list(df_unweighted_bi_direct_degree_centrality) if "V" in h]].std(axis=1)
df_unweighted_bi_direct_degree_centrality_temp = df_unweighted_bi_direct_degree_centrality[[h for h in list(df_unweighted_bi_direct_degree_centrality) if "bi_direct_degree_centrality_V" in h]]
df_unweighted_bi_direct_degree_centrality_temp.reset_index(drop=True, inplace = True)
df_unweighted_bi_direct_degree_centrality["unweighted_bi_direct_degree_centrality_delta_survey"] = delta_survey(df_unweighted_bi_direct_degree_centrality_temp)
df_unweighted_bi_direct_degree_centrality["unweighted_bi_direct_degree_centrality_rel_difference"] = rel_diff_survey(df_unweighted_bi_direct_degree_centrality_temp)

df_weighted_bi_direct_degree_centrality["fr_weighted_bi_direct_degree_centrality_mean"] = df_weighted_bi_direct_degree_centrality[[h for h in list(df_weighted_bi_direct_degree_centrality) if "V" in h]].mean(axis=1)
df_weighted_bi_direct_degree_centrality["fr_weighted_bi_direct_degree_centrality_var"] = df_weighted_bi_direct_degree_centrality[[h for h in list(df_weighted_bi_direct_degree_centrality) if "V" in h]].var(axis=1)
df_weighted_bi_direct_degree_centrality["fr_weighted_bi_direct_degree_centrality_std"] = df_weighted_bi_direct_degree_centrality[[h for h in list(df_weighted_bi_direct_degree_centrality) if "V" in h]].std(axis=1)
df_weighted_bi_direct_degree_centrality_temp = df_weighted_bi_direct_degree_centrality[[h for h in list(df_weighted_bi_direct_degree_centrality) if "fr_weighted_bi_direct_degree_centrality_V" in h]]
df_weighted_bi_direct_degree_centrality_temp.reset_index(drop=True, inplace = True)
df_weighted_bi_direct_degree_centrality["fr_weighted_bi_direct_degree_centrality_delta_survey"] = delta_survey(df_weighted_bi_direct_degree_centrality_temp)
df_weighted_bi_direct_degree_centrality["fr_weighted_bi_direct_degree_centrality_rel_difference"] = rel_diff_survey(df_weighted_bi_direct_degree_centrality_temp)

In [ ]:
df_weighted_bi_direct_degree_centrality.head()

In [ ]:
df_ego_followers_density = pd.read_excel(base_path + "Analysis/ego_followers_density.xlsx") 
df_ego_followers_density.head()

In [ ]:
df_ego_friends_density = pd.read_excel(base_path + "Analysis/ego_friends_density.xlsx")
df_ego_friends_density.head()

In [ ]:
df_ego_bi_direct_density = pd.read_excel(base_path + "Analysis/ego_bi_direct_density.xlsx") 
df_ego_bi_direct_density.head()

# Friends to Follower Ratio

In [ ]:
df_friends_to_followers = df_startupsfollowers_degree[["company_name","followers_degree_mean"]].merge(df_startupsfriends_degree[["company_name","friends_degree_mean"]], on='company_name', how='left')
df_friends_to_followers['followers_div_by_friends_degree_ratio'] = df_friends_to_followers['followers_degree_mean']/df_friends_to_followers['friends_degree_mean']
df_friends_to_followers = df_friends_to_followers.replace([np.inf, -np.inf], np.nan)
df_friends_to_followers.dropna(inplace = True)
df_friends_to_followers.drop(columns=['followers_degree_mean', 'friends_degree_mean'], inplace = True)
print(df_friends_to_followers.shape)
df_friends_to_followers

In [ ]:
df_company2investor = pd.read_excel(base_path + "Analysis/SNA/SNA investors and companies.xlsx", 'V14df_company_SNA',skiprows=0,usecols = "A,C,D,E,F")
df_company2investor.head()

# Create matches with most connected twitter_users

In [ ]:
df_most_matches_with_most_connected_followers = pd.read_excel(base_path + "Analysis/max_matches_followers.xlsx")
df_most_matches_with_most_connected_followers.head()

In [ ]:
df_most_matches_with_most_connected_friends = pd.read_excel(base_path + "Analysis/max_matches_friends.xlsx")
df_most_matches_with_most_connected_friends.head()

In [ ]:
df_most_matches_with_most_connected_bi_direct = pd.read_excel(base_path + "Analysis/max_matches_bi_direct.xlsx")
df_most_matches_with_most_connected_bi_direct.head()

# Create Events Dataframe (Independent Variable)

Import Events

In [ ]:
df_events = pd.read_excel(base_path + "Pitchbook_Crunchbase_Tracxn_Raw/Don't touch " + version_id + "/" + version_id + "_Pitchbook_Crunchbase_Tracxn_Database Crunchbase funding added.xlsx" , version_id + '_event', skiprows=0, usecols="A,B,C,D")
df_events = df_events.loc[df_events['name_change'] != "name change"]
df_events.drop(['name_change'], axis=1, inplace = True)
df_events = df_events[~df_events.event_type.str.contains("before")]

# #------------------------Clean company_name:------------------------#
# df_events['company_name'] = df_events['company_name'].str.replace(r"\(.*\)","")
# df_events['company_name'] = df_events['company_name'].str.replace("ltd.","")
# df_events['company_name'] = df_events['company_name'].str.replace("ltd","")
# df_events['company_name'] = df_events['company_name'].str.replace("Ltd.","")
# df_events['company_name'] = df_events['company_name'].str.replace("Ltd","")
# df_events['company_name'] = df_events['company_name'].str.replace("LTD","")
# df_events['company_name'] = df_events['company_name'].str.replace("LLC","")
# df_events['company_name'] = df_events['company_name'].str.replace('Limited','')
# df_events['company_name'] = df_events['company_name'].str.lower()
# df_events.replace('^\s+', '', regex=True, inplace=True) #front
# df_events.replace('\s+$', '', regex=True, inplace=True) #end

# df_events_new_column_company_name_dict = []
# for i in range(len(df_events['company_name'])):
#     element = df_events['company_name'][i]
#     if df_events['company_name'][i] in company_name_dict:
#         df_events_new_column_company_name_dict.append(company_name_dict[element])
#     else:
#         df_events_new_column_company_name_dict.append(df_events['company_name'][i])
        
# df_events['company_name'] = df_events_new_column_company_name_dict
# #-----------------------------------------------------------#

# #--------------------------------------Dropper:--------------------------------------#
# df_events['dropper'] = 'active'
# df_events_new_column_dropper = []
# for i in range(len(df_events['company_name'])):
#     element = df_events['company_name'][i]
#     if df_events['company_name'][i] in dropper:
#         df_events_new_column_dropper.append(dropper[element])
#     else:
#         df_events_new_column_dropper.append(df_events['dropper'][i])
# df_events['dropper'] = df_events_new_column_dropper

# df_events = df_events[df_events.dropper != 'inactive']
# df_events = df_events[df_events.dropper != 'late stage']
# df_events = df_events[df_events.dropper != 'not ai startup']
# df_events = df_events[df_events.dropper != 'acquired']
# df_events = df_events[df_events.dropper != 'debtfinance']
# df_events = df_events[df_events.dropper != 'PEbacked']
# df_events = df_events[df_events.dropper != 'duplicate']
# #-------------------------------------------------------------#

# print('After dropping we have ' + '\033[1m' + str(len(df_events.index)) + '\033[0m' + ' companies left in the Events Dataset')
df_events.head()

In [ ]:
# df_events_show = df_events.loc[df_events['event_type'] == str("Founded during study")]
# df_events_show.head()

 Create a deal counter function to create lists with elements in range

In [ ]:
companies_list = df_events["company_name"].tolist()
companies_set = sorted(list(set(companies_list)))
print(companies_set[:10])

Create list with deal counts

In [ ]:
company_counts = []
for company in companies_set:
    count_company = companies_list.count(company)
    company_counts.append(count_company)
    
company_max_events_counts = []
for company_count in company_counts:
    l = []
    l += company_count * [company_count]
    company_max_events_counts.extend(l)

df_events["company_max_events_counts"] = company_max_events_counts
df_events.head()

Spread columns out based on how many events occurred

In [ ]:
# Get total max event numbers
company_max_events_counts = np.max(df_events.company_max_events_counts.values)
#print("Max deal numbers ", company_max_events_counts)

# Create header for result dataframe

header = ["company_name"]
for i in range(company_max_events_counts):
  header.append("event_type"+str(i))
  header.append("event_date"+str(i))
  
# define result dataframe
distributed_events_dataframe = pd.DataFrame(columns=header)

# populate result dataframe
for company in list(df_events.company_name.unique()):
  sub_data = df_events.loc[df_events.loc[:,"company_name"]==company]
  sub_data.reset_index(inplace=True, drop=True)
  
  value_list = [company] + [None for i in range(company_max_events_counts*2)] 
  #print(value_list)
  for row in range(len(sub_data)):
    
    
    event_type = sub_data.loc[row, "event_type"]
    event_date = sub_data.loc[row, "event_date"]

    
    value_list[row*2+1] = event_type
    value_list[row*2+2] = event_date

    
  distributed_events_dataframe = distributed_events_dataframe.append(pd.DataFrame([value_list], columns=header))
  

distributed_events_dataframe.reset_index(inplace=True, drop=True)
distributed_events_dataframe['had_event'] = True

distributed_events_dataframe.head()

Concatenate with all companies dataframe and fill had_event = False when there was no event

Create event number column

In [ ]:
def createList(r_end): 
    return [item for item in range(1, r_end + 1)] 

In [ ]:
event_number = []
for count in company_counts:
    event_number.extend(createList(count))
    
df_events["event_number"] = event_number

df_events.head()

Spread columns out based on how many events occurred

In [ ]:
# Get total max event numbers
company_max_events_counts = np.max(df_events.company_max_events_counts.values)
#print("Max deal numbers ", company_max_events_counts)

# Create header for result dataframe

header = ["company_name"]
for i in range(company_max_events_counts):
  header.append("event_type"+str(i))
  header.append("event_date"+str(i))
  
# define result dataframe
distributed_events_dataframe = pd.DataFrame(columns=header)

# populate result dataframe
for company in list(df_events.company_name.unique()):
  sub_data = df_events.loc[df_events.loc[:,"company_name"]==company]
  sub_data.reset_index(inplace=True, drop=True)
  
  value_list = [company] + [None for i in range(company_max_events_counts*2)] 
  #print(value_list)
  for row in range(len(sub_data)):
    
    
    event_type = sub_data.loc[row, "event_type"]
    event_date = sub_data.loc[row, "event_date"]

    
    value_list[row*2+1] = event_type
    value_list[row*2+2] = event_date

    
  distributed_events_dataframe = distributed_events_dataframe.append(pd.DataFrame([value_list], columns=header))
  

distributed_events_dataframe.reset_index(inplace=True, drop=True)
distributed_events_dataframe['had_event'] = True

distributed_events_dataframe.head()

# Concatenate all dataframes

Concatenate with all companies dataframe and fill had_event = False when there was no event

In [ ]:
df_result = company_name_df
df_result = df_result.merge(df_funding_rounds_and_deltas, on='company_name', how='left')
df_result = df_result.merge(df_log_total_raised_to_date, on='company_name', how='left')
df_result = df_result.merge(df_log_max_post_money_valuation, on='company_name', how='left')

df_result = df_result.merge(df_company_names_in_versions, on='company_name', how='left')
df_result = df_result.merge(df_twitter_ids_in_versions, on='company_name', how='left')
df_result = df_result.merge(df_twitter_handles_in_versions, on='current_twitter_id', how='left')

df_result = df_result.merge(df_company_websites_no_duplicates_no_nan, on='company_name', how='left')

df_result = df_result.merge(df_accelerated, on='company_name', how='left')
df_result = df_result.merge(df_invested, on='company_name', how='left')

df_result = df_result.merge(src_pitchbook_df, on='company_name', how='left')
df_result = df_result.merge(src_crunchbase_df, on='company_name', how='left')
df_result = df_result.merge(src_tracxn_df, on='company_name', how='left')

df_result = df_result.merge(df_webrank, on='company_name', how='left')

df_result = df_result.merge(df_company_tweet_stats, on='company_name', how='left')

df_result = df_result.merge(df_b2b, on='company_name', how='left')

df_result = df_result.merge(df_hq_city, on='company_name', how='left')

df_result = df_result.merge(df_startupsfnf_degree, on='company_name', how='left')
df_result = df_result.merge(df_startupsfnf_degree_centrality, on='company_name', how='left')
df_result = df_result.merge(df_startupsfriends_degree, on='company_name', how='left')
df_result = df_result.merge(df_startupsfriends_degree_centrality, on='company_name', how='left')
df_result = df_result.merge(df_startupsfollowers_degree, on='company_name', how='left')
df_result = df_result.merge(df_startupsfollowers_degree_centrality, on='company_name', how='left')
df_result = df_result.merge(df_unweighted_bi_direct_degree, on='company_name', how='left')
df_result = df_result.merge(df_unweighted_bi_direct_degree_centrality, on='company_name', how='left')
df_result = df_result.merge(df_weighted_bi_direct_degree, on='company_name', how='left')
df_result = df_result.merge(df_weighted_bi_direct_degree_centrality, on='company_name', how='left')
df_result = df_result.merge(df_friends_to_followers, on='company_name', how='left')

df_result = df_result.merge(df_ego_followers_density, on='company_name', how='left')
df_result = df_result.merge(df_ego_friends_density, on='company_name', how='left')
df_result = df_result.merge(df_ego_bi_direct_density, on='company_name', how='left')

df_result = df_result.merge(df_company2investor, on='company_name', how='left')

df_result = df_result.merge(df_most_matches_with_most_connected_followers, on='company_name', how='left')
df_result = df_result.merge(df_most_matches_with_most_connected_friends, on='company_name', how='left')
df_result = df_result.merge(df_most_matches_with_most_connected_bi_direct, on='company_name', how='left')

df_result = df_result.merge(df_new_to_followon_investor_ratio, on='company_name', how='left')

df_result = df_result.merge(df_followers_sum_all_users, on='company_name', how='left')
df_result["followers_match_count_per_user"] = df_result['followers_match_count_mean']/df_result['followers_degree_mean']
df_result = df_result.merge(df_followers_total_matched_words, on='company_name', how='left')
df_result = df_result.merge(df_friends_sum_all_users, on='company_name', how='left')
df_result["friends_match_count_per_user"] = df_result['friends_match_count_mean']/df_result['friends_degree_mean']
df_result = df_result.merge(df_friends_total_matched_words, on='company_name', how='left')
df_result = df_result.merge(df_fnf_sum_all_users, on='company_name', how='left')
df_result["unweighted_bi_direct_match_count_per_user"] = df_result['unweighted_bi_direct_match_count_mean']/df_result['unweighted_bi_direct_degree_mean']
df_result = df_result.merge(df_fnf_total_matched_words, on='company_name', how='left')

df_result = df_result.merge(distributed_events_dataframe, on='company_name', how='left')

df_result['invested'] = df_result['invested'].fillna(False)
df_result['accelerated'] = df_result['accelerated'].fillna(False)
df_result['had_event'] = df_result['had_event'].fillna(False)
df_result['has_twitter'] = df_result["has_twitter"].fillna(False)
df_result['has_website'] = df_result["has_website"].fillna(False)

df_result.head()

## Manual edits

Clean non AI companies

In [ ]:
df_result = df_result[(df_result['company_name'] != "aurora-ai")]
df_result = df_result[(df_result['company_name'] != "react")]

df_result.reset_index(drop=True, inplace = True)

Clean invested missing

In [ ]:
df_result.loc[df_result.company_name == "droneid", "invested"] = True
df_result.loc[df_result.company_name == "goinstore", "invested"] = True
df_result.loc[df_result.company_name == "mapadore", "invested"] = True
df_result.loc[df_result.company_name == "perchpeek", "invested"] = True
df_result.loc[df_result.company_name == "pobuca", "invested"] = True
df_result.loc[df_result.company_name == "policycastle", "invested"] = True
df_result.loc[df_result.company_name == "quorso", "invested"] = True
df_result.loc[df_result.company_name == "droneid", "invested"] = True
df_result.loc[df_result.company_name == "vet-ai", "invested"] = True
df_result.loc[df_result.company_name == "chatterbox", "invested"] = True
df_result.loc[df_result.company_name == "topolytics", "invested"] = True
df_result.loc[df_result.company_name == "wt infotech", "invested"] = True

Clean b2b missing

In [ ]:
df_result.loc[df_result.company_name == "aa research and analytics", "b2b"] = 1

In [ ]:
df_result_temp = df_result[['company_name','b2b']]
df_result_temp[0:12]

In [ ]:
df_result_temp.loc[df_result_temp['company_name'] == "aa research and analytics"]

Add missing events

In [ ]:
df_result.loc[df_result.company_name == "typescore", "deal_date_2"] = datetime.datetime(2019, 2, 5)
df_result.loc[df_result.company_name == "typescore", "deal_type_2"] = "Out of Business"

df_result.loc[df_result.company_name == "exantas", "deal_date_2"] = datetime.datetime(2019, 1, 10)
df_result.loc[df_result.company_name == "exantas", "deal_type_2"] = "Out of Business"

In [ ]:
df_result = df_result.loc[df_result['current_twitter_handle'] != "aventuspf"]

In [ ]:
df_result.loc[df_result['company_name'] == "aventus systems"]

In [ ]:
df_result["company_name"].value_counts().max()

In [ ]:
twitter_id_headers = [e for e in list(df_result) if "twitter_id" in e and e not in ('dropper')]
twitter_id_headers
df_result[twitter_id_headers] = df_result[twitter_id_headers].fillna(0).astype(int).astype(str).replace("0", "NA")
df_result.head()

# Describe

## Overview

In [ ]:
df_result_describe = df_result.describe()
df_result_describe

## Deciles

In [ ]:
df_result_deciles = df_result.quantile([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
df_result_deciles

### Fill missing cells

In [ ]:
#df_database.loc[df_database.company_name == "10x banking", "twitter_handle_1"] = "LALALA TWITTER HANDLE"

# Save

In [ ]:
with pd.ExcelWriter(base_path + "Analysis/Database.xlsx") as writer:
  df_result.to_excel(writer, sheet_name= "df_database", index=False)
  df_result_describe.to_excel(writer, sheet_name= "df_result_describe")
  df_result_deciles.to_excel(writer, sheet_name= "df_result_deciles")
   
print("File save to " + base_path + "Analysis/Database.xlsx " + str(os.path.isfile(base_path + "Analysis/Database.xlsx")) + " File check")